In [1]:
from pydna.dseq import Dseq
from pydna.dseqrecord import Dseqrecord
from pydna.assembly import Assembly


forward_primer = "TTACGTAAAAAA"
reverse_primer = "CGTACGTTTTTT"
overlap_length = 6
overhang = overlap_length - len(forward_primer)

# Using Dseq, we can assemble the two oligos automatically
templateless = Dseq(forward_primer, reverse_primer, ovhg=overhang)
templateless



Dseq(-18)
TTACGTAAAAAA
      TTTTTTGCATGC

In [2]:
templateless = templateless.fill_in()
dseq = Dseqrecord(templateless,
                  id="tpless",
                  name="Templateless PCR",
                  description="Templateless PCR")


# Using the original assembly structure
fwd = Dseqrecord(Dseq(forward_primer))
rvs = Dseqrecord(Dseq(reverse_primer))

Assembly((fwd, rvs), limit=6)
dseq.seq


Dseq(-18)
TTACGTAAAAAAcgtacg
aatgcaTTTTTTGCATGC

In [3]:
rvs.seq

Dseq(-12)
CGTACGTTTTTT
GCATGCAAAAAA

In [4]:
a = Dseqrecord(Dseq("ATCGGG"))
b = Dseqrecord(Dseq("GGGTCA"))

Assembly((fwd,rvs), limit=6)

Assembly
fragments..: 12bp 12bp
limit(bp)..: 6
G.nodes....: 2
algorithm..: common_sub_strings

In [5]:
# Using assembly2 functions

from assembly2 import Assembly as Assembly2
from assembly2 import TemplatelessPCRAssembly
from assembly2 import assembly2str_tuple

f = Assembly2((fwd, rvs), limit=6)

for ass in f.get_linear_assemblies():
    print(assembly2str_tuple(ass))

print()

g = TemplatelessPCRAssembly((fwd, rvs), limit=6)

for ass in g.get_linear_assemblies():
    print(assembly2str_tuple(ass))



((1, -2, '[6:12]', '[0:6]'),)
((-1, 2, '[0:6]', '[6:12]'),)

((1, -2, '[6:12]', '[0:6]'),)


In [9]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from dna_functions import format_sequence_genbank, read_dsrecord_from_json
from main import app
from fastapi.testclient import TestClient
from pydna.parsers import parse as pydna_parse
from Bio.Restriction.Restriction import CommOnly
from pydantic_models import (
    PCRSource,
    TemplatelessPCRSource,
    PrimerModel,
    SequenceEntity,

)
from pydna.dseqrecord import Dseqrecord
from pydna.dseq import Dseq

client = TestClient(app)

def test_templateless():
    submitted_source = TemplatelessPCRSource(
        input=[],
        forward_primer=1,
        reverse_primer=2,
    )

    # Test case of initial example

    primer_fwd = PrimerModel(sequence="TTACGTAAAAAA", id=1, name="fwd")
    primer_rvs = PrimerModel(sequence="CGTACGTTTTTT", id=2, name="rvs")

    data = {
        'source': submitted_source.model_dump(),
        'sequences': [],
        'primers': [primer_fwd.model_dump(), primer_rvs.model_dump()],
    }

    response = client.post('/templateless_pcr', json=data, params={'minimal_annealing': 6})
    payload = response.json()

    sources = [TemplatelessPCRSource.model_validate(s) for s in payload['sources']]
    sequences = [read_dsrecord_from_json(SequenceEntity.model_validate(s)) for s in payload['sequences']]

    assert len(sources) == 1
    assert len(sequences) == 1

    seq1 = sequences[0]
    assert str(seq1.seq) == "TTACGTAAAAAACGTACG"

    # Test case of fully overlapping primers (it is not particularly special)

    primer_fwd = PrimerModel(sequence="ACGTACGT", id=1, name="fwd")
    primer_rvs = PrimerModel(sequence="ACGTACGT", id=2, name="rvs")

    data = {
        'source': submitted_source.model_dump(),
        'sequences': [],
        'primers': [primer_fwd.model_dump(), primer_rvs.model_dump()],
    }

    response = client.post('/templateless_pcr', json=data, params={'minimal_annealing': 8})
    payload = response.json()

    sources = [TemplatelessPCRSource.model_validate(s) for s in payload['sources']]
    sequences = [read_dsrecord_from_json(SequenceEntity.model_validate(s)) for s in payload['sequences']]

    assert len(sources) == 1
    assert len(sequences) == 1

    seq1 = sequences[0]
    assert str(seq1.seq) == "ACGTACGT"

    # Test several sources
    response = client.post('/templateless_pcr', json=data, params={'minimal_annealing': 4})
    payload = response.json()

    sources = [TemplatelessPCRSource.model_validate(s) for s in payload['sources']]
    sequences = [read_dsrecord_from_json(SequenceEntity.model_validate(s)) for s in payload['sequences']]

    assert len(sources) == 2
    assert len(sequences) == 2

    seq1 = sequences[0]
    seq2 = sequences[1]

    assert str(seq1.seq) == "ACGTACGT"
    assert str(seq2.seq) == "ACGTACGTACGT"

    # Test overhanging primers (should result in a fragment with overhang, as this would happen in a real PCR)
    primer_fwd = PrimerModel(sequence="ACGTACGT", id=1, name="fwd")
    primer_rvs = PrimerModel(sequence="ACGTACGTTTT", id=2, name="rvs")

    data = {
        'source': submitted_source.model_dump(),
        'sequences': [],
        'primers': [primer_fwd.model_dump(), primer_rvs.model_dump()],
    }

    response = client.post('/templateless_pcr', json=data, params={'minimal_annealing': 8})
    payload = response.json()

    sources = [TemplatelessPCRSource.model_validate(s) for s in payload['sources']]
    sequences = [read_dsrecord_from_json(SequenceEntity.model_validate(s)) for s in payload['sequences']]

    assert len(sources) == 1
    assert len(sequences) == 1

    seq1 = sequences[0]
    print(seq1)


test_templateless()

Dseqrecord
circular: False
size: 8
ID: None
Name: name
Description: description
Number of features: 0
/molecule_type=DNA
/topology=linear
/data_file_division=UNK
/date=01-JAN-1980
/accessions=['id']
/keywords=['']
/source=
/organism=.
/taxonomy=[]
Dseq(-8)
ACGTACGT
TGCATGCA


In [12]:
ds = Dseq("ACGTACGT", "ACGTACGTTTT")
ds = Dseqrecord(ds, id="tpless", name="Templateless PCR", description="Templateless PCR")
print(ds)

Dseqrecord
circular: False
size: 11
ID: tpless
Name: Templateless PCR
Description: Templateless PCR
Number of features: 0
/molecule_type=DNA
Dseq(-11)
   ACGTACGT
TTTTGCATGCA


In [17]:
from pydna.dseq import Dseq
from pydna.dseqrecord import Dseqrecord
from pydna.assembly import Assembly

ass = Assembly((Dseqrecord(Dseq("ACGTACGT")), Dseqrecord(Dseq("ACGTACGTTTT"))), limit=8)

for a in ass.assemble_linear():
    print(a)

Dseqrecord
circular: False
size: 11
ID: id
Name: name
Description: description
Number of features: 0
/molecule_type=DNA
Dseq(-11)
ACGTACGTTTT
TGCATGCAAAA
Dseqrecord
circular: False
size: 8
ID: id
Name: name
Description: description
Number of features: 0
/molecule_type=DNA
Dseq(-8)
ACGTACGT
TGCATGCA


In [22]:
Assembly((Dseqrecord("GCGTGACTCCgttttagagctagaaatagcaagttaaaataa"), Dseqrecord("TTTTGCAAAGAttcttcggtacaggttatgttttttggcaaca")))

for a in ass.assemble_linear():
    print(a)

Dseqrecord
circular: False
size: 11
ID: id
Name: name
Description: description
Number of features: 0
/molecule_type=DNA
Dseq(-11)
ACGTACGTTTT
TGCATGCAAAA
Dseqrecord
circular: False
size: 8
ID: id
Name: name
Description: description
Number of features: 0
/molecule_type=DNA
Dseq(-8)
ACGTACGT
TGCATGCA
